In [11]:
import urllib

DATA_ROOT = "data"
DATA_URL = "https://raw.githubusercontent.com/tidyverse/ggplot2/main/data-raw/diamonds.csv"

DATA_FILEPATH = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(DATA_URL, DATA_FILEPATH)

('data\\data.csv', <http.client.HTTPMessage at 0x1e60fc93a00>)

In [12]:
import pandas as pd

df = pd.read_csv("data/data.csv")
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [2]:
df.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [3]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

### Set Variable

In [14]:
PIPELINE_NAME = "diamonds-pipeline"
SCHEMA_PIPELINE_NAME = "diamonds-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

In [15]:
DATA_ROOT = "data"

In [16]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

### ExampleGen

In [17]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base="data", output_config=output)
interactive_context.run(example_gen)


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 3
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [19]:
from pprint import PrettyPrinter
pp = PrettyPrinter()

train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

tfrecord_train_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_train_filenames, compression_type='GZIP')

for tfrecord in dataset.take(2):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "\"carat\""
    value {
      float_list {
        value: 0.23000000417232513
      }
    }
  }
  feature {
    key: "\"clarity\""
    value {
      bytes_list {
        value: "SI2"
      }
    }
  }
  feature {
    key: "\"color\""
    value {
      bytes_list {
        value: "E"
      }
    }
  }
  feature {
    key: "\"cut\""
    value {
      bytes_list {
        value: "Ideal"
      }
    }
  }
  feature {
    key: "\"depth\""
    value {
      float_list {
        value: 61.5
      }
    }
  }
  feature {
    key: "\"price\""
    value {
      int64_list {
        value: 326
      }
    }
  }
  feature {
    key: "\"table\""
    value {
      float_list {
        value: 55.0
      }
    }
  }
  feature {
    key: "\"x\""
    value {
      float_list {
        value: 3.950000047683716
      }
    }
  }
  feature {
    key: "\"y\""
    value {
      float_list {
        value: 3.9800000190734863
      }
    }
  }
  feature {
    key: "\"z\""
    va

### StatisticGen

In [20]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)

interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 4
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [21]:
interactive_context.show(statistics_gen.outputs["statistics"])

### SchemaGen

In [22]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 5
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [23]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
"'""carat""'",FLOAT,required,,-
"'""clarity""'",STRING,required,,"'""clarity""'"
"'""color""'",STRING,required,,"'""color""'"
"'""cut""'",STRING,required,,"'""cut""'"
"'""depth""'",FLOAT,required,,-
"'""price""'",INT,required,,-
"'""table""'",FLOAT,required,,-
"'""x""'",FLOAT,required,,-
"'""y""'",FLOAT,required,,-


,Values
Domain,
"'""clarity""'","'I1', 'IF', 'SI1', 'SI2', 'VS1', 'VS2', 'VVS1', 'VVS2'"
"'""color""'","'D', 'E', 'F', 'G', 'H', 'I', 'J'"
"'""cut""'","'Fair', 'Good', 'Ideal', 'Premium', 'Very Good'"


### ExampleValidator

In [24]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 6
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [25]:
interactive_context.show(example_validator.outputs['anomalies'])

### Transform

In [ ]:
TRANSFORM_MODULE_FILE = "_transform.py"

In [26]:
%%writefile {"_transform.py"}

import tensorflow as tf
import tensorflow_transform as tft


NUMERICAL_FEATURES = ["depth", "carat", "table", "x", "y", "z"]
CATEGORICAL_FEATURES = ["clarity", "color", "cut"]

LABEL_KEY = "price"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}
    
    for feature in NUMERICAL_FEATURES:
        outputs[transformed_name(feature)] = tft.scale_to_z_score(inputs[feature])
    
    for feature in CATEGORICAL_FEATURES:
        outputs[transformed_name(feature)] = tft.compute_and_apply_vocabulary(inputs[feature])
    
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Overwriting _transform.py


: 

In [ ]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file="_transform.py"
)

interactive_context.run(transform)

### Set MetadataStore